In [7]:
import os
import json
import requests
import delta
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, ArrayType
import env.config

In [8]:
def get_spark() -> SparkSession:
    builder = (
        pyspark.sql.SparkSession.builder
        .master("spark://Subhashs-MacBook-Pro.local:7077")
        .appName("spark")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
    )
    spark = delta.configure_spark_with_delta_pip(builder).getOrCreate()
    spark.sparkContext.setLogLevel("ERROR")
    return spark

if (SparkSession.getActiveSession()):
    print('Spark session is already active, using that..')
else:
    print('Starting a new spark session in cluster mode..')
    spark = get_spark()

Starting a new spark session in cluster mode..


In [9]:
def call_data_gov_api(uri):
    """
    """

    url = "https://api.data.gov.in"
    api_key = env.config.api_key
    resp_format = "json"
    limit = 500
    offset = 0
    resp = requests.get("{}{}?api-key={}&format={}&limit={}&offset={}".format(url,uri,api_key,resp_format,limit,offset))

    return resp

In [10]:
dir = os.getcwd()
spark = get_spark()

In [11]:
uri = "/resource/81153f15-b4da-45b5-a299-f307351c5001"
resp = call_data_gov_api(uri)
resp.json()['records']

json_array_schema = StructType([
    StructField('sno', StringType(), nullable=False), 
    StructField('memberid', StringType(), nullable=False),
    StructField('name', StringType(), nullable=False),
    StructField('party', StringType(), nullable=False),
    StructField('constituencies', StringType(), nullable=False),
    StructField('state', StringType(), nullable=False), 
    StructField('paddress', StringType(), nullable=False),
    StructField('permanentphone', StringType(), nullable=False),
    StructField('localaddress', StringType(), nullable=False),
    StructField('localphone', StringType(), nullable=False),
    StructField('emailid', StringType(), nullable=False)
  ])

# Read as a dictionary
df = spark.createDataFrame(data=resp.json()['records'], schema = json_array_schema)

In [12]:
# # The following  only stores data in delta file format
# df.write.format("delta").mode("overwrite").save(dir + os.sep + "/data/ls_members")
df.show(truncate=True)

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/Users/subhashpeshwa/Library/Python/3.9/lib/python/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/subhashpeshwa/Library/Python/3.9/lib/python/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [18]:
import shutil
shutil.rmtree('/Users/subhashpeshwa/Desktop/Databricks/CountryWatch-Databricks/Local/spark-warehouse/ls_members')

In [19]:
table_name = "ls_members"
df.write.mode("overwrite").saveAsTable(table_name)

In [20]:
display(spark.sql('DESCRIBE DETAIL default.ls_members'))

DataFrame[format: string, id: string, name: string, description: string, location: string, createdAt: timestamp, lastModified: timestamp, partitionColumns: array<string>, numFiles: bigint, sizeInBytes: bigint, properties: map<string,string>, minReaderVersion: int, minWriterVersion: int, tableFeatures: array<string>]

In [21]:
spark.sql("SHOW DATABASES;").show()
spark.sql("SHOW TABLES;").show()

+---------+
|namespace|
+---------+
|  default|
+---------+

+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|  default|ls_members|      false|
+---------+----------+-----------+



In [23]:
spark.sql("SELECT * FROM default.ls_members;").show()

+---+--------+--------------------+------+-----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|sno|memberid|                name| party|   constituencies|           state|            paddress|      permanentphone|        localaddress|          localphone|             emailid|
+---+--------+--------------------+------+-----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|187|    4744|Karadi, Shri Sang...|   BJP|           Koppal|       Karnataka|Flat No. 54/63, T...|Telefax: (08539) ...|Flat No. 2 &amp; ...|                 N/A|  bjpdpkpl@gmail.com|
|188|    4616|Karandlaje, Km. S...|   BJP|Udupi Chikmagalur|       Karnataka|Karandlaje House,...|Tel : (08251) 284...|101, Narmada, , M...|Tel : (011) 23717975|shobha.karandlaje...|
|189|    4178|Karunakaran, Shri P.|CPI(M)|        Kasaragod|          Kerala|Amaravat

In [18]:
spark.sql("DROP TABLE default.ls_members;").show()

++
||
++
++



In [ ]:
db